# MNIST Database MLP optimization with Numerical Methods!

## Getting the data from the Built in MNIST Data set in tensorflow Module

In [ ]:
import numpy as np
import tensorflow as tf


def load_and_preprocess_mnist():
    # Carregando as Imagens
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()   # Dataset do Tensorflow

    # Normalizando as Imagens...
    threshold = 128
    train_images_flattened = np.where(train_images > threshold, 1, 0).reshape(train_images.shape[0], -1)
    test_images_flattened = np.where(test_images > threshold, 1, 0).reshape(test_images.shape[0], -1)

    return (train_images_flattened, train_labels)  # Retornando array Principal

# Exemplo de Uso...
(train_images_flattened, train_labels) = load_and_preprocess_mnist()




11490434/11490434 [==============================] - 2s 0us/step


In [ ]:

# Example: Print the first flattened image and its label
#print("First training image (flattened):", train_images_flattened[200])
#n = int(input("Enter a number of the index up to 60k: "))
#print("Label of the first training image:", train_labels[n])
print("----------------------------------------")
#print(f"The data set is {len(train_images_flattened)} long:"

----------------------------------------


# ---------------------------------------------------------------------

## Making a Pygame Grid That Reconstruct the Image!

# ---------------------------------------------------------------------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # GPU

xs = [torch.tensor(x, dtype=torch.double, device=device) for x in train_images_flattened]
xs = torch.stack(xs)
ys = torch.tensor(train_labels, dtype=torch.long, device=device)


class Brein(nn.Module):
    def __init__(self):
        super(Brein, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)   # wn * xn + bn  - camada Linear (28*28)in -> 512 neutonios
        self.relu = nn.ReLU()              # Funcao ReLu de Ativacao
        self.fc2 = nn.Linear(512, 10)      # Camada Linear - (512)in -> 10 out

    def forward(self, x):   # Foward Pass
        x = self.fc1(x)     # Passando nas Respectivas Camadas!!
        x = self.relu(x)
        x = self.fc2(x)
        return x

def train_model(train_data, train_labels, model, criterion, optimizer, epochs=20, batch_size=128, tol=10e-3):

    # Convertendo Dados Para Tensors e carregando em GPU
    train_data = torch.tensor(train_data, dtype=torch.float32, device=device)
    train_labels = torch.tensor(train_labels, dtype=torch.long, device=device)
    train_dataset = TensorDataset(train_data, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    big_loop = 0
    #for loop in range(big_loop):
    mode = 0

    loss = 1000   # Inicializando Loss...(condicao do loop)
    while loss > tol:
        for data, labels in train_loader:
            optimizer.zero_grad()    # Grad = 0
            outputs = model(data)    # Foward pass
            loss = criterion(outputs, labels)    # Loss function ...
            loss.backward()
            optimizer.step()
        print(f'Iteration {big_loop+1}, Loss: {loss.item()}')

        big_loop += 1
    return model

# Initialize the model
model = Brein().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Prepare your MNIST data
(train_images_flattened, train_labels) = load_and_preprocess_mnist()

# Train the model
trained_model = train_model(train_images_flattened, train_labels, model, criterion, optimizer, epochs=20, tol=10e-8)

# Function to predict the digit
def predict_digit(image, model):
    image = torch.tensor(image, dtype=torch.float32)
    outputs = model(image.unsqueeze(0))  # Add batch dimension
    _, predicted = torch.max(outputs, 1)
    return predicted.item()



Iteration 1, Loss: 0.2534589469432831
Iteration 2, Loss: 0.16589994728565216
Iteration 3, Loss: 0.13893385231494904
Iteration 4, Loss: 0.1037982702255249
Iteration 5, Loss: 0.03833826258778572
Iteration 6, Loss: 0.0632224977016449
Iteration 7, Loss: 0.02839057147502899
Iteration 8, Loss: 0.010263952426612377
Iteration 9, Loss: 0.0161778274923563
Iteration 10, Loss: 0.0018627290846779943
Iteration 11, Loss: 0.018230142071843147
Iteration 12, Loss: 0.0016934688901528716
Iteration 13, Loss: 0.002214412670582533
Iteration 14, Loss: 0.023281188681721687
Iteration 15, Loss: 0.00586911803111434
Iteration 16, Loss: 0.001206003944389522
Iteration 17, Loss: 0.0003790329210460186
Iteration 18, Loss: 0.0004245871677994728
Iteration 19, Loss: 0.00016722333384677768
Iteration 20, Loss: 0.00011454011109890416
Iteration 21, Loss: 4.266729229129851e-05
Iteration 22, Loss: 0.00042301148641854525
Iteration 23, Loss: 8.202128083212301e-05
Iteration 24, Loss: 9.974862769013271e-05
Iteration 25, Loss: 3.102

In [ ]:
# Testando
image = xs[0] # Primeira Imagem do banco de Dados
predicted_digit = predict_digit(image, trained_model)
print(f'Predicted Digit: {predicted_digit}')
print(f'Predicted Digit: {train_labels[0]}')

Predicted Digit: 5
Predicted Digit: 5


<ipython-input-3-018f982367c9>:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32)


In [ ]:
torch.save(model.state_dict(), 'mnist_model.pth')